# German General Personas

TODO @Jens add Descriptions, explanations, etc.

## Imports

In [1]:
# General Imports
import pandas as pd
import requests
import zipfile
import io

# Either local inference with vllm or remote with AsyncOpenAI
from openai import AsyncOpenAI

In [2]:
# qstn Imports
from qstn.survey_manager import conduct_survey_single_item
from qstn.parser import parse_json, raw_responses
from qstn.utilities import create_one_dataframe

from qstn.prompt_builder import LLMPrompt, generate_likert_options
from qstn.utilities import placeholder
from qstn.utilities import AnswerOptions

from qstn.inference import response_generation

In [3]:
SEED = 42

## Preparing the Dataset

We will load the personas from the github.

In [4]:
PERSONAS_TO_LOAD = 20

In [5]:
zip_url = "https://github.com/germanpersonas/German-General-Personas/raw/main/GGP_all_topk_fulltext.zip"

response = requests.get(zip_url)
response.raise_for_status()  # Check for errors

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("pc_fulltext_sociodemographics_only.jsonl") as f:
        df_personas = pd.read_json(f, lines=True, nrows=PERSONAS_TO_LOAD)

df_personas = df_personas.rename(columns={0: "persona"})
print(f"Loaded {len(df_personas)} rows.")
print(df_personas.head())

Loaded 20 rows.
                                             persona
0  Du bist eine Person, 56 Jahre alt, wohnhaft in...
1  Du bist eine Person, 71 Jahre alt, männlich un...
2  Du bist eine Person, 80 Jahre alt, weiblich, d...
3  Du bist eine Person, 52 Jahre alt und weiblich...
4  Du bist eine Person, 57 Jahre alt, weiblich un...


In [6]:
# @Jens, if you load this into the Github or somewhere publicly we can also load this on the fly, which would probably look nicer

json_questionnaire = {
  "lp04": {
    "task_type": "question",
    "statement": "Sind Sie bei der folgenden Aussage derselben oder anderer Meinung: 'So wie die Zukunft aussieht, kann man es kaum noch verantworten, Kinder auf die Welt zu bringen.'?",
    "answers": [
      "1: BIN DERS.MEINUNG",
      "2: BIN ANDERER MEINUNG"
    ]
  },
  "pe05": {
    "task_type": "question",
    "statement": "Inwiefern stimmen Sie der folgenden Meinung zu: 'Die Politiker bemühen sich im Allgemeinen darum, die Interessen der Bevölkerung zu vertreten.'? (Antwortmöglichkeiten: voll und ganz zustimmen, eher zustimmen, eher nicht zustimmen, überhaupt nicht zustimmen)",
    "answers": [
      "1: STIMME VOLL ZU",
      "2: STIMME EHER ZU",
      "3: STIMME EHER NICHT ZU",
      "4: STIMME GAR NICHT ZU"
    ]
  },
  "mp18": {
    "task_type": "question",
    "statement": "Ergeben sich Ihrer Meinung nach wegen der Flüchtlinge in Bezug auf das Zusammenleben in der Gesellschaft mehr Chancen, mehr Risiken oder weder noch?",
    "answers": [
      "1: RISIKO UEBERWIEGT",
      "2: EHER RISIKO",
      "3: WEDER NOCH",
      "4: EHER CHANCE",
      "5: CHANCE UEBERWIEGT"
    ]
  },
  "mm01": {
    "task_type": "question",
    "statement": "Inwieweit stimmen Sie der folgenden Aussage zu: 'Die Ausübung des islamischen Glaubens in Deutschland sollte eingeschränkt werden.'?",
    "answers": [
      "1: 1 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "2: 2 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "3: 3 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "4: 4 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "5: 5 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "6: 6 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "7: 7 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")"
    ]
  },
  "vi10": {
    "task_type": "question",
    "statement": "Wie wichtig ist es für Sie persönlich 'sich politisch zu engagieren'?",
    "answers": [
      "1: 1 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "2: 2 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "3: 3 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "4: 4 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "5: 5 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "6: 6 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "7: 7 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")"
    ]
  },
  "vm17": {
    "task_type": "question",
    "statement": "Wie beurteilen Sie es, wenn ein Paar anonym gespendete Ei- oder Samenzellen verwendet, um mit medizinischer Hilfe ein Kind zu bekommen?",
    "answers": [
      "1: -3 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "2: -2 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "3: -1 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "4: 0 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "5: 1 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "6: 2 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "7: 3 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")"
    ]
  },
  "st01": {
    "task_type": "question",
    "statement": "Manche Leute sagen, dass man den meisten Menschen trauen kann. Andere meinen, dass man nicht vorsichtig genug sein kann im Umgang mit anderen Menschen. Was ist Ihre Meinung dazu?",
    "answers": [
      "1: MAN KANN TRAUEN",
      "2: MUSS VORSICHTIG SEIN",
      "3: KOMMT DARAUF AN",
      "4: SONSTIGES"
    ]
  },
  "iw04": {
    "task_type": "question",
    "statement": "Inwiefern stimmen Sie der folgenden Auffassung zu: 'Der Staat muss dafür sorgen, dass man auch bei Krankheit, Not, Arbeitslosigkeit und im Alter ein gutes Auskommen hat.'? (Antwortmöglichkeiten: voll zustimmen, eher zustimmen, eher nicht zustimmen, überhaupt nicht zustimmen)",
    "answers": [
      "1: STIMME VOLL ZU",
      "2: STIMME EHER ZU",
      "3: STIMME EHER NICHT ZU",
      "4: STIMME GAR NICHT ZU"
    ]
  },
  "ep06": {
    "task_type": "question",
    "statement": "Was glauben Sie, wie wird Ihre eigene wirtschaftliche Lage in einem Jahr sein?",
    "answers": [
      "1: WESENTLICH BESSER",
      "2: ETWAS BESSER",
      "3: GLEICHBLEIBEND",
      "4: ETWAS SCHLECHTER",
      "5: WESENTL. SCHLECHTER"
    ]
  },
  "li03": {
    "task_type": "question",
    "statement": "Wie wichtig ist für Sie der Lebensbereich 'Freizeit und Erholung'? Bitte antworten Sie auf einer Skala von 1 (unwichtig) bis 7 (sehr wichtig).",
    "answers": [
      "1: 1 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "2: 2 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "3: 3 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "4: 4 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "5: 5 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "6: 6 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "7: 7 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")"
    ]
  },
  "ps01": {
    "task_type": "question",
    "statement": "Wie zufrieden sind Sie - insgesamt betrachtet - mit den gegenwärtigen Leistungen der Bundesregierung?",
    "answers": [
      "1: SEHR ZUFRIEDEN",
      "2: ZIEMLICH ZUFRIEDEN",
      "3: ETWAS ZUFRIEDEN",
      "4: ETWAS UNZUFRIEDEN",
      "5: ZIEML. UNZUFRIEDEN",
      "6: SEHR UNZUFRIEDEN"
    ]
  },
  "mp12": {
    "task_type": "question",
    "statement": "Inwieweit stimmen Sie der folgenden Aussage zu: 'Die Ausländer in Deutschland tragen dazu bei, den Fachkräftemangel zu beheben.'?",
    "answers": [
      "1: 1 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "2: 2 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "3: 3 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "4: 4 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "5: 5 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "6: 6 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "7: 7 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")"
    ]
  },
  "rb01": {
    "task_type": "question",
    "statement": "Wie stark stimmen Sie der folgenden Aussage zu: 'Es gibt einen Gott, der sich mit jedem Menschen persönlich befasst.'?",
    "answers": [
      "1: STIMME VOLL ZU",
      "2: STIMME EHER ZU",
      "3: KEINE FESTE MEINUNG",
      "4: STIMME EHER NICHT ZU",
      "5: STIMME GAR NICHT ZU",
      "6: NOCH NIE NACHGEDACHT"
    ]
  },
  "vi06": {
    "task_type": "question",
    "statement": "Wie wichtig ist es für Sie persönlich 'sozial Benachteiligten und gesellschaftlichen Randgruppen zu helfen'?",
    "answers": [
      "1: 1 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "2: 2 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "3: 3 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "4: 4 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "5: 5 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "6: 6 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "7: 7 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")"
    ]
  },
  "ca12": {
    "task_type": "question",
    "statement": "Für wie schlimm halten Sie persönlich das folgende Verhalten: Jemand raucht mehrmals in der Woche Haschisch. (Antwortmöglichkeiten: sehr schlimm, ziemlich schlimm, weniger schlimm, überhaupt nicht schlimm)",
    "answers": [
      "1: SEHR SCHLIMM",
      "2: ZIEMLICH SCHLIMM",
      "3: WENIGER SCHLIMM",
      "4: GAR NICHT SCHLIMM"
    ]
  },
  "sm01": {
    "task_type": "question",
    "statement": "Sind Sie derzeit Mitglied in einer Gewerkschaft?",
    "answers": [
      "1: JA",
      "2: NEIN"
    ]
  },
  "im08": {
    "task_type": "question",
    "statement": "Wie wichtig ist Ihrer Meinung nach 'Leistung, Fleiß' für den Aufstieg in unserer Gesellschaft gegenwärtig?",
    "answers": [
      "1: SEHR WICHTIG",
      "2: WICHTIG",
      "3: WENIGER WICHTIG",
      "4: UNWICHTIG"
    ]
  },
  "ep03": {
    "task_type": "question",
    "statement": "Wie beurteilen Sie Ihre eigene wirtschaftliche Lage heute?",
    "answers": [
      "1: SEHR GUT",
      "2: GUT",
      "3: TEILS/TEILS",
      "4: SCHLECHT",
      "5: SEHR SCHLECHT"
    ]
  },
  "lp03": {
    "task_type": "question",
    "statement": "Sind Sie bei der folgenden Aussage derselben oder anderer Meinung: 'Egal was manche Leute sagen: Die Situation der einfachen Leute wird nicht besser, sondern schlechter.'?",
    "answers": [
      "1: BIN DERS.MEINUNG",
      "2: BIN ANDERER MEINUNG"
    ]
  },
  "pt15": {
    "task_type": "question",
    "statement": "Wie groß ist Ihr Vertrauen in die politischen Parteien? (Skala: 1=überhaupt kein Vertrauen, 7=sehr großes Vertrauen)",
    "answers": [
      "1: 1 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")",
      "2: 2 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")",
      "3: 3 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")",
      "4: 4 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")",
      "5: 5 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")",
      "6: 6 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")",
      "7: 7 (1-7 \"GAR KEIN VERTRAUEN\"-\"GROSSES VERTRAUEN\")"
    ]
  },
  "mn01": {
    "task_type": "question",
    "statement": "Wie wichtig sollte bei der Vergabe der deutschen Staatsbürgerschaft sein, ob die Person in Deutschland geboren ist?",
    "answers": [
      "1: 1 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")",
      "2: 2 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")",
      "3: 3 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")",
      "4: 4 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")",
      "5: 5 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")",
      "6: 6 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")",
      "7: 7 (1-7 \"GAR NICHT WICHTIG\"-\"SEHR WICHTIG\")"
    ]
  },
  "mm06": {
    "task_type": "question",
    "statement": "Inwieweit stimmen Sie der folgenden Aussage zu: 'Ich habe den Eindruck, dass unter den in Deutschland lebenden Muslimen viele religiöse Fanatiker sind.'?",
    "answers": [
      "1: 1 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "2: 2 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "3: 3 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "4: 4 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "5: 5 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "6: 6 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")",
      "7: 7 (1-7 \"STIMME GAR NICHT ZU\"-\"STIMME VOLL+GANZ ZU\")"
    ]
  },
  "vi07": {
    "task_type": "question",
    "statement": "Wie wichtig ist es für Sie persönlich 'sich und seine Bedürfnisse gegen andere durchzusetzen'?",
    "answers": [
      "1: 1 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "2: 2 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "3: 3 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "4: 4 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "5: 5 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "6: 6 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")",
      "7: 7 (1-7 \"UNWICHTIG\"-\"SEHR WICHTIG\")"
    ]
  },
  "vm16": {
    "task_type": "question",
    "statement": "Wie beurteilen Sie es, wenn ein Paar eigene Ei- und Samenzellen verwendet, um mit medizinischer Hilfe ein Kind zu bekommen?",
    "answers": [
      "1: -3 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "2: -2 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "3: -1 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "4: 0 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "5: 1 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "6: 2 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")",
      "7: 3 (\"-3\"-\"+3\" \"SEHR FALSCH\"-\"SEHR RICHTIG\")"
    ]
  },
  "sm02": {
    "task_type": "question",
    "statement": "Waren Sie früher einmal Mitglied in einer Gewerkschaft?",
    "answers": [
      "1: JA",
      "2: NEIN"
    ]
  },
  "im03": {
    "task_type": "question",
    "statement": "Wie wichtig ist Ihrer Meinung nach 'Bildung, Ausbildung' für den Aufstieg in unserer Gesellschaft gegenwärtig?",
    "answers": [
      "1: SEHR WICHTIG",
      "2: WICHTIG",
      "3: WENIGER WICHTIG",
      "4: UNWICHTIG"
    ]
  },
  "ep01": {
    "task_type": "question",
    "statement": "Wie beurteilen Sie ganz allgemein die heutige wirtschaftliche Lage in Deutschland?",
    "answers": [
      "1: SEHR GUT",
      "2: GUT",
      "3: TEILS/TEILS",
      "4: SCHLECHT",
      "5: SEHR SCHLECHT"
    ]
  }
}

In [7]:
json_questionnaire["mp18"]

{'task_type': 'question',
 'statement': 'Ergeben sich Ihrer Meinung nach wegen der Flüchtlinge in Bezug auf das Zusammenleben in der Gesellschaft mehr Chancen, mehr Risiken oder weder noch?',
 'answers': ['1: RISIKO UEBERWIEGT',
  '2: EHER RISIKO',
  '3: WEDER NOCH',
  '4: EHER CHANCE',
  '5: CHANCE UEBERWIEGT']}

We extract the questions.

In [8]:
questionnaire_list = []

for key, value in json_questionnaire.items():
    # Create a new empty dict for this row
    questionnaire_item = {}
    
    # Update it with the specific format you wanted
    questionnaire_item.update({
        "questionnaire_item_id": key, 
        "question_content": value["statement"]
    })
    
    # Add to the list
    questionnaire_list.append(questionnaire_item)
questionnaire = pd.DataFrame(questionnaire_list)

In [9]:
print(questionnaire.head(3))

  questionnaire_item_id                                   question_content
0                  lp04  Sind Sie bei der folgenden Aussage derselben o...
1                  pe05  Inwiefern stimmen Sie der folgenden Meinung zu...
2                  mp18  Ergeben sich Ihrer Meinung nach wegen der Flüc...


We extract the answers and get only their descriptions.

In [10]:
all_cleaned_answers = []
for key, value in json_questionnaire.items():
    cleaned_answers = []
    
    for i, answer in enumerate(value["answers"]):
        clean_text = answer.split(": ")[1] 
        
        # We simply check if the text contains a minus -> If it does it is a from to scale
        if "-" in clean_text:
            from_to_scale = True
            # We only want to have name index
            #clean_text = clean_text.split('(')
            cleaned_answers.append(clean_text)
        else:
            from_to_scale = False
            cleaned_answers.append(clean_text)
    
    all_cleaned_answers.append({"question": key, "answer": cleaned_answers, "from_to_scale": from_to_scale})

In [11]:
all_cleaned_answers[2]

{'question': 'mp18',
 'answer': ['RISIKO UEBERWIEGT',
  'EHER RISIKO',
  'WEDER NOCH',
  'EHER CHANCE',
  'CHANCE UEBERWIEGT'],
 'from_to_scale': False}

## System Prompt, User Prompt and Personas

In [12]:
system_prompt = "Nehme die Perspektive der folgenden Person ein: {persona}"
prompt = (
    f"Welche der Antwortmöglichkeiten ist die Reaktion der Person auf folgende Frage: {placeholder.PROMPT_QUESTIONS}\n"
    f"{placeholder.PROMPT_OPTIONS}\n"
    f"{placeholder.PROMPT_AUTOMATIC_OUTPUT_INSTRUCTIONS}"
)

## Different Ways to get the output

In [13]:
OUTPUT_METHODS = ["OPEN", "RESTRICTED_CHOICE", "REASONING_JSON", "VERBALIZED_DISTRIBUTION"]
# Select your method here by copying one from above
output_method = "RESTRICTED_CHOICE"

### Creating the LLM Prompts

In [14]:
def create_llm_prompts(row: pd.Series):
    persona_index = row.name
    persona_str = row["persona"]

    # We create a LLMPrompt for each persona
    llm_prompt = LLMPrompt(
        questionnaire_source=questionnaire,
        questionnaire_name=str(persona_index),
        system_prompt=system_prompt.format(persona=persona_str),
        prompt=prompt
    )

    # Here we define how the LLM should answer the question
    answer_options = {}
    
    for dic in all_cleaned_answers:
        answers = dic["answer"]
        from_to_scale = dic["from_to_scale"]
        rgm: response_generation.ResponseGenerationMethod = None

        
        # We change the ResponseGenerationMethod here. All other code stays the same
        if output_method == "OPEN":
            pass            
        elif output_method == "RESTRICTED_CHOICE":
            rgm = response_generation.ChoiceResponseGenerationMethod(answers, output_template=f"Antworte nur mit der exakten Antwort.")
        elif output_method == "REASONING_JSON":
            rgm = response_generation.JSONReasoningResponseGenerationMethod(output_template=f"Antworte nur im folgenden JSON format:\n{placeholder.JSON_TEMPLATE}")
        elif output_method == "VERBALIZED_DISTRIBUTION":
            rgm = response_generation.JSONVerbalizedDistribution(output_template=f"Gib für jede Antwortmöglichkeit eine Wahrscheinlichkeit an, mit der die Person antwortet. Nutze dafür folgendes JSON format:\n{placeholder.JSON_TEMPLATE}")

        # We can check for robustness with generate_likert_options: 
        # Randomized or reversed options order, different indeces etc.
        if from_to_scale:
            answer_option = generate_likert_options(n=len(answers), answer_texts=answers, only_from_to_scale=True, scale_prompt_template="Antwortmöglichkeiten: {start} bis {end}", response_generation_method=rgm)
        else:
            answer_option = generate_likert_options(n=len(answers), answer_texts=answers, list_prompt_template="Antwortmöglichkeiten: {options}" , response_generation_method=rgm)
        answer_options[dic["question"]]= answer_option

    llm_prompt.prepare_prompt(answer_options=answer_options)
    return llm_prompt

llm_prompts: list[LLMPrompt] = df_personas.apply(create_llm_prompts, axis=1).to_list()

In [15]:
sys_prompt, user_prompt = llm_prompts[0].get_prompt_for_questionnaire_type(item_id="mp18")
print("SYSTEM:", sys_prompt)
print("USER:", user_prompt)

SYSTEM: Nehme die Perspektive der folgenden Person ein: Du bist eine Person, 56 Jahre alt, wohnhaft in Westdeutschland in einer kleinen bis mittelgroßen Stadt. Du hast die Mittlere Reife als höchsten Schulabschluss und bist deutsche Staatsangehörige. Dein monatliches Nettoeinkommen liegt zwischen 1000 und 1124 Euro. Aktuell arbeitest du als Verkäuferin in Handelsgeschäften und hast die berufliche Stellung einer Arbeiterin inne. Du hast keinen Universitätsabschluss.
USER: Welche der Antwortmöglichkeiten ist die Reaktion der Person auf folgende Frage: Ergeben sich Ihrer Meinung nach wegen der Flüchtlinge in Bezug auf das Zusammenleben in der Gesellschaft mehr Chancen, mehr Risiken oder weder noch?
Antwortmöglichkeiten: 1: RISIKO UEBERWIEGT, 2: EHER RISIKO, 3: WEDER NOCH, 4: EHER CHANCE, 5: CHANCE UEBERWIEGT
Antworte nur mit der exakten Antwort.


## Inference

In [16]:
model_id = "Qwen/Qwen3-VL-4B-Instruct"

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

generator = AsyncOpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [17]:
results = conduct_survey_single_item(
    generator, llm_prompts=llm_prompts, client_model_name=model_id, max_tokens=2000, seed=SEED
)

Processing questionnaires:   0%|          | 0/27 [00:00<?, ?it/s]

/home/maxi/Documents/QSTN/src/qstn/inference/remote_inference.py:229: UserWarning: Strict Choice Response Generation is only supported for vllm APIs.
  warnings.warn("Strict Choice Response Generation is only supported for vllm APIs.")
Processing Prompts: 100%|██████████| 20/20 [00:00<00:00, 23.41it/s]
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retrying request to /chat/completions in 0.420072 seconds
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retrying request to /chat/completions in 0.496874 seconds
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retrying request to /chat/completions in 0.465621 seconds
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retrying request to /chat/completions in 0.472099 seconds
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retrying request to /chat/completions in 0.407941 seconds
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retrying request to /chat/completions in 0.415413 seconds
[2026-01-22 14:50:22] INFO _base_client.py:1618: Retry

In [18]:
# If we expect JSON output we can automatically parse it
if output_method == "REASONING_JSON" or output_method == "VERBALIZED_DISTRIBUTION":
    parsed_results = parse_json(results)
else:
    parsed_results = raw_responses(results)

full_results = create_one_dataframe(parsed_results)
full_results.head()

,questionnaire_name,questionnaire_item_id,question,llm_response,logprobs,reasoning
0,0,lp04,Sind Sie bei der folgenden Aussage derselben o...,BIN ANDERER MEINUNG,None,None
1,0,pe05,Inwiefern stimmen Sie der folgenden Meinung zu...,STIMME EHER NICHT ZU,None,None
2,0,mp18,Ergeben sich Ihrer Meinung nach wegen der Flüc...,RISIKO UEBERWIEGT,None,None
3,0,mm01,Inwieweit stimmen Sie der folgenden Aussage zu...,"1 (1-7 ""STIMME GAR NICHT ZU""-""STIMME VOLL+GANZ...",None,None
4,0,vi10,Wie wichtig ist es für Sie persönlich 'sich po...,"4 (1-7 ""UNWICHTIG""-""SEHR WICHTIG"")",None,None
